In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cProfile

import pyaccel, sirius
import mathphys as mp
%load_ext autoreload
%autoreload 1
%aimport lattice_errors
np.random.seed(19880419)

In [ ]:
acc      = sirius.SI_V10.create_accelerator()
fam_data = sirius.SI_V10.get_family_data(acc)

In [ ]:
um, mrad, percent = 1e-6, 1e-3, 1e-2
config = dict({'mags':dict(),'girder':dict()})

config['mags']['quads'] = dict({'labels':list(),'sigma':dict()})
config['mags']['quads']['labels'] = ['qfa','qda','qdb1','qdb2','qfb','qf1','qf2','qf3','qf4']
config['mags']['quads']['sigma']['x']     = 40 * um * 1
config['mags']['quads']['sigma']['y']     = 40 * um * 1
config['mags']['quads']['sigma']['roll']  = 0.30 * mrad * 1
config['mags']['quads']['sigma']['excit'] = 0.05 * percent * 1

config['mags']['sexts'] = dict({'labels':list(),'sigma':dict()})
config['mags']['sexts']['labels'] = ['sfa','sda','sf1j','sd1j','sd2j','sd3j','sf2j',
                                     'sdb','sfb','sf1k','sd1k','sd2k','sd3k','sf2k']
config['mags']['sexts']['sigma']['x']     = 40 * um * 1
config['mags']['sexts']['sigma']['y']     = 40 * um * 1
config['mags']['sexts']['sigma']['roll']  = 0.30 * mrad * 1
config['mags']['sexts']['sigma']['excit'] = 0.05 * percent * 1

config['mags']['bc'] = dict({'labels':list(),'sigma':dict()})
config['mags']['bc']['labels'] = ['bc_lf','bc_hf']
config['mags']['bc']['sigma']['x']     = 40 * um * 1
config['mags']['bc']['sigma']['y']     = 40 * um * 1
config['mags']['bc']['sigma']['roll']  = 0.30 * mrad * 1
config['mags']['bc']['sigma']['excit'] = 0.05 * percent * 1
config['mags']['bc']['sigma']['k_dip'] = 0.10 * percent * 1

config['mags']['bendblocks']['labels'] = ['b1','b2']
config['mags']['bendblocks'].nrsegs       = [1,1]
config['mags']['bendblocks']['sigma']['x']      = 40 * um * 1
config['mags']['bendblocks']['sigma']['y']      = 40 * um * 1
config['mags']['bendblocks']['sigma']['roll']   = 0.30 * mrad * 1
config['mags']['bendblocks']['sigma']['excit']  = 0.05 * percent * 1
config['mags']['bendblocks']['sigma']['k_dip']  = 0.10 * percent * 1
                                  
errors = lattice_errors.generate_errors('name',acc,config,fam_data=fam_data,nr_mach=20,cutoff=1,rndtype='gauss')

In [ ]:
plt.plot(errors['x'][:6].T,'.')
plt.show()

In [ ]:
config['girder']['x']     = 80 * um * 1
config['girder']['y']     = 80 * um * 1;
config['girder']['roll']  =0.2 * mrad * 1;

errors = lattice_errors.generate_errors('name',acc,config,fam_data=fam_data,nr_mach=20,cutoff=1,rndtype='gauss')

In [ ]:
plt.plot(errors['x'][:6].T,'.')
plt.show()

In [ ]:
machine = lattice_errors.apply_erros('name',acc,errors,increment=1.0)

In [ ]:
a = [[1,3],[3,4],[2,3]]
b = [sum([a[i][j] for j in range(len(a[i]))]) for i in range(len(a))]
b

In [ ]:
if 5%2: print('not ok')

In [ ]:
np.zeros((2,2))

In [ ]:
m66, matr = pyaccel.tracking.findm66(acc)

In [ ]:
cProfile.run('tw, *_ = pyaccel.optics.calc_twiss(acc)')

In [ ]:
betax,betay = pyaccel.optics.get_twiss?

In [ ]:
betax,betay,spos = pyaccel.optics.get_twiss(tw,['betax','betay','spos'])

In [ ]:
plt.plot(spos,betax)
plt.plot(spos,betay)
plt.show()